Check if GPU is available (enable GPU through: Runtime-->Change runtime type --> select "GPU" in hardware accelerator dropdown list)

In [2]:
import torch
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3060 Laptop GPU


import package

In [3]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
from sample.compute import *
from load_data import *


d:\Anaconda\envs\torch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def extract_wifi_rssi(all_data):
    wifi_rssi = {}
    for position_key in all_data:
        wifi_ds = all_data[position_key]['wifi']
        for wifi_d in wifi_ds:
            bssid = wifi_d[1]
            rssi = int(wifi_d[2])

            if bssid in wifi_rssi:
                position_rssi = wifi_rssi[bssid]
                if position_key in position_rssi:
                    rssi = position_rssi[position_key][0]
                    wifi_count = position_rssi[position_key][1]
                    position_rssi[position_key][0] = (rssi * (wifi_count + 1)) / (wifi_count + 1)
                    position_rssi[position_key][1] += 1
                else:
                    position_rssi[position_key] = [rssi, 1]
            else:
                position_rssi = {position_key: [rssi, 1]}

            wifi_rssi[bssid] = position_rssi

    return wifi_rssi

In [5]:
def extract_ibeacon_rssi(all_data):
    ibeacon_rssi = {}
    for position_key in all_data:

        ibeacon_ds = all_data[position_key]['ibeacon']
        for ibeacon_d in ibeacon_ds:
            ummid = ibeacon_d[1]
            rssi = int(ibeacon_d[2])

            if ummid in ibeacon_rssi:
                position_rssi = ibeacon_rssi[ummid]
                if position_key in position_rssi:
                    old_rssi = position_rssi[position_key][0]
                    old_count = position_rssi[position_key][1]
                    position_rssi[position_key][0] = (old_rssi * old_count + rssi) / (old_count + 1)
                    position_rssi[position_key][1] = old_count + 1
                else:
                    position_rssi[position_key] = np.array([rssi, 1])
            else:
                position_rssi = {}
                position_rssi[position_key] = np.array([rssi, 1])

            ibeacon_rssi[ummid] = position_rssi

    return ibeacon_rssi

In [8]:
def extract_data(files_path, augmentation=True, one_position=False, use_wifi=True, use_ibeacon=True):
    import csv

    all_data = combine_data_with_position(files_path, augmentation)

    wifi_rssi = extract_wifi_rssi(all_data)
    ibeacon_rssi = extract_ibeacon_rssi(all_data)
    
    wifi_bssid = list(wifi_rssi.keys())
    ibeacon_ummid = list(ibeacon_rssi.keys())

    wifi_bssid_length = 0
    ibeacon_ummid_length = 0
    if one_position:
        begin_idx = 2
        index = ['position', 'magn_strength']
    else:
        begin_idx = 3
        index = ['position_x', 'position_y', 'magn_strength']

    if use_wifi:
        wifi_bssid_length = len(wifi_bssid)
        index.extend(wifi_bssid)
    
    if use_ibeacon:
        ibeacon_ummid_length = len(ibeacon_ummid)
        index.extend(ibeacon_ummid)

    data_file_path = '../data/data_for_train.csv'
    with open(data_file_path, newline='', mode='w') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(index)

        for position_key in all_data:

            row_data = []

            # write position data
            if one_position:
                row_data.append(str(position_key))
            else:
                row_data.append(position_key[0])
                row_data.append(position_key[1])

            # write magnetic strength
            magnetic_data = all_data[position_key]['magnetic']
            magn_stength = np.mean(np.sqrt(np.sum(magnetic_data[:, 1:4] ** 2, axis=1)))
            row_data.append(magn_stength)

            # write wifi rssi
            if use_wifi:
                for bssid in index[begin_idx: begin_idx + wifi_bssid_length]:
                    if position_key in wifi_rssi[bssid]:
                        row_data.append(wifi_rssi[bssid][position_key][0])
                    else:
                        row_data.append(0)

            # write ibeacon rssi
            if use_ibeacon:
                for ummid in index[begin_idx + wifi_bssid_length: begin_idx + wifi_bssid_length+ibeacon_ummid_length]:
                    if position_key in ibeacon_rssi[ummid]:
                        row_data.append(ibeacon_rssi[ummid][position_key][0])
                    else:
                        row_data.append(0)


            csv_writer.writerow(row_data)
        
        print('The csv file is completed')
    

In [9]:
files_path = os.path.join('../data/', 'site1', 'F1', 'path_data_files')
extract_data(files_path, one_position=False, use_wifi=True, use_ibeacon=True)

Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7aac5b77e0006b1732b.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7abc5b77e0006b1732d.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7b29191710006b5705b.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7b7c5b77e0006b1732f.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7ba9191710006b5705d.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7bdc5b77e0006b17331.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7bf9191710006b5705f.txt
Processing file: E:\course\urban_computing\assignment\project_1\data\site1\F1\path_data_files\5dd9e7c1c5b77e0006b17333.txt
Processing file: